In [45]:
import reservoirpy as rpy
import numpy as np

# 1. Set some global parameters
rpy.verbosity(0)  # no need to be too verbose here
rpy.set_seed(42)  # make everything reproducible!

In [46]:
# パラメータ設定
n_timesteps = 5000
units = 2000
spectral_radii = np.linspace(0.1, 1.5, 10)
num_esn_instances = lime = 10
inter_sparsity = 0.25

In [47]:
# 2. Create the reservoir and readout
from reservoirpy.nodes import Reservoir, Ridge

def create_model(spectral_radius):
    reservoir = Reservoir(units=units, sr=spectral_radius)
    readout = Ridge(ridge=1e-7)
    return reservoir >> readout

In [48]:
def create_Standard_esn(sr, units):
    reservoir = Reservoir(units=units, input_scaling=1,lr=0.6, sr=sr)
    readout = Ridge(output_dim=1, ridge=1e-5)
    return reservoir >> readout

def create_4modular_esn(sr, n_modules=4):
    modules = [Reservoir(units=units//n_modules, input_scaling=1,sr=sr) for _ in range(n_modules)]
    readout = Ridge(output_dim=1, ridge=1e-5)
    return [modules[0],modules[1],modules[2],modules[3]] >> readout

def create_4modular_esn_diff(sr, n_modules=4):
    modules0 = Reservoir(units=units//n_modules, rc_connectivity=0.2, sr=sr)
    modules1 = Reservoir(units=units//n_modules, input_scaling=0.6, rc_connectivity=0.4, sr=sr)
    modules2 = Reservoir(units=units//n_modules, input_scaling=0.9, W=uniform, rc_connectivity=0.6, sr=sr)
    modules3 = Reservoir(units=units//n_modules, input_scaling=0.3, W=bernoulli, rc_connectivity=0.8, sr=sr)
    readout = Ridge(output_dim=1, ridge=1e-5)
    return [modules0,modules1,modules2,modules3] >> readout

def create_4modular_esn_diff_inter(sr, n_modules=4):
    modules0 = Reservoir(units=units//n_modules, rc_connectivity=0.2, sr=sr)
    modules1 = Reservoir(units=units//n_modules, input_scaling=0.6, rc_connectivity=0.4, inter_connectivity=inter_sparsity, sr=sr)
    modules2 = Reservoir(units=units//n_modules, input_scaling=0.9, W=uniform, rc_connectivity=0.6, inter_connectivity=inter_sparsity,sr=sr)
    modules3 = Reservoir(units=units//n_modules, input_scaling=0.3, W=bernoulli, rc_connectivity=0.8, inter_connectivity=inter_sparsity, sr=sr)
    readout = Ridge(output_dim=1, ridge=1e-5)
    return [modules0,modules1,modules2,modules3] >> readout

In [49]:
rmse_values = {"Standard": [], "4_Modules": [],  "4_Hetero_Modules": [],  "4_Hetero_Modules_inter": []}
std_values = {"Standard": [], "4_Modules": [],  "4_Hetero_Modules": [],  "4_Hetero_Modules_inter": []}
results = {"Standard": [], "4_Modules": [],  "4_Hetero_Modules": [],  "4_Hetero_Modules_inter": []}
for _ in tqdm(range(num_esn_instances), desc="Testing ESN instances"):
    instance_rmse = {"Standard": [], "4_Modules": [],  "4_Hetero_Modules": [],  "4_Hetero_Modules_inter": []}
    for sr in spectral_radii:
        
        esn_4modular_diff = create_4modular_esn_diff(sr)
        esn_4modular_diff.fit(X1[:2000], X1[1:2001], warmup=100)
        predictions_4modular_diff = esn_4modular_diff.run(X2[2001:-1])
        rmse_4modular_diff = np.sqrt(mean_squared_error(X2[2002:], predictions_4modular_diff))
       
        esn_4modular_diff_inter = create_4modular_esn_diff(sr)
        esn_4modular_diff_inter.fit(X1[:2000], X1[1:2001], warmup=100)
        predictions_4modular_diff_inter = esn_4modular_diff_inter.run(X2[2001:-1])
        rmse_4modular_diff_inter = np.sqrt(mean_squared_error(X2[2002:], predictions_4modular_diff_inter))       
     
        esn_4modular = create_4modular_esn(sr)
        esn_4modular.fit(X1[:2000], X1[1:2001], warmup=100)
        predictions_4modular = esn_4modular.run(X2[2001:-1])
        rmse_4modular = np.sqrt(mean_squared_error(X2[2002:], predictions_4modular))
        
        Standard_esn = create_Standard_esn(sr,100)
        Standard_esn.fit(X1[:2000], X1[1:2001], warmup=100)
        Standard_predictions = Standard_esn.run(X2[2001:-1])
        Standard_rmse = np.sqrt(mean_squared_error(X2[2002:], Standard_predictions))
        
        instance_rmse["4_Hetero_Modules"].append(min(rmse_4modular_diff*lime, 1))
        instance_rmse["4_Hetero_Modules_inter"].append(min(rmse_4modular_diff_inter*lime, 1))
        instance_rmse["Standard"].append(min(Standard_rmse*lime, 1))
        instance_rmse["4_Modules"].append(min(rmse_4modular*lime, 1))
mean_rmse_values = {key: np.mean(val, axis=1) for key, val in results.items()}
std_rmse_values = {key: np.std(val, axis=1) for key, val in results.items()}

NameError: name 'tqdm' is not defined

In [ ]:
plt.figure(figsize=(15, 8))
for key in mean_rmse_values:
    plt.errorbar(spectral_radii, mean_rmse_values[key], yerr=std_rmse_values[key], label=f"{key.replace('_', ' ').title()} ", fmt='o-', capsize=5)

plt.title('Mean RMSE vs Spectral Radius in Echo State Networks with Error Bars')
plt.xlabel('Spectral Radius')
plt.ylabel('Mean RMSE')
plt.legend()
plt.grid(True)
plt.show()

data_file_path = os.path.join(data_folder_path, "final_result.pkl")
with open(data_file_path, 'wb') as f:
    pickle.dump((mean_rmse_values, std_rmse_values), f)
print("Results saved to 'final_result.pkl'")